# AR3,4

*09/11/2023*

* Exposure patterns for sample AR3 and AR4.

## Imports & Organisation

In [67]:
import math
import numpy as np
import klayout.db as db

from pathlib import Path
from IPython.display import Markdown, display

root = Path().absolute()
out_path = Path(f'{root}/Data')

## Voyager Settings

* We start by setting up the grating and chip size for the voyager.
* These remain pretty constant during the exposure, so we can set them as variables at the top.
* The grating size I will use is 500 $\mu m$, and the chip size is 15 x 15mm.

In [68]:
writefield_height = 1000
writefield_width = 1000
chip_height = 15000
chip_width = 15000

* We assume that the database layout will be in $\mu m$, so the above numbers are set to these units.
* Later we ensure that the database units are set accordingly.

## Database Setup

* Here is where we set the database units to $\mu m$

In [69]:
layout = db.Layout()
layout.dbu = 0.001
sample_identifier = 'AR4'
top_cell = layout.create_cell(f'{sample_identifier}')

## Functions

* To keep things simple, we can write some of the grating generation as functions.
* Simple things such as making a grating, or a nanohole array.

In [70]:
def makes_bars_cell(layout : object,
                    layer : object,
                    period : float,
                    fill_factor : float,
                    bar_height : float,
                    bar_identifier : str) -> object:
    """
    Create bar cell with the given dimensions.

    Parameters
    ---------
    layout, layer: object
        Database and layer objects from KLayout.
    period, fill_factor, bar_height: float
        Period, fill factor, and height of the bar.
    bar_identifier: string
        Bar cell name.

    Returns
    -------
    bar_cell: object
        Bar cell.

    See Also
    --------
    None

    Notes
    -----
    None

    Example
    -------
    None

    """
    bar_cell = layout.create_cell(f'{bar_identifier}')
    bar_origin = db.DPoint(0, 0)
    bar = db.DBox(
        bar_origin,
        (bar_origin + db.DVector(period * (1 - fill_factor), bar_height)))
    bar_cell.shapes(layer).insert(bar)
    return bar_cell


def makes_hole_cell(layout : object,
                    layer : object,
                    radius : float,
                    number_of_vertices : int,
                    hole_identifier : str) -> object:
    """
    Create hole cell with given dimensions.

    Parameters
    ----------
    layout, layer: object
        Database and layer objects from KLayout.
    radius: float
        Hole radius in database units.
    number_of_vertices: int
        Number of polygon vertices.
    hole_identifier: string
        Hole cell name.
    
    Returns
    -------
    hole_cell: object
        Hole cell.
    
    See Also
    --------
    makes_bars_cell

    Notes
    -----
    None

    Example
    -------
    None

    """
    hole_cell = layout.create_cell(f'{hole_identifier}')
    hole = db.DPolygon.ellipse(
        db.DBox(0 - radius, 0 - radius, 0 + radius, 0 + radius),
        number_of_vertices)
    hole_cell.shapes(layer).insert(hole)
    return hole_cell


def generates_texts(layout : object,
                    layer_index : int,
                    dose : float,
                    text_identifier : str,
                    text_string : str,
                    text_magnification : int):
    """
    Generate text string cell in KLayout.

    Parameters
    ---------
    layout: object
        Database object from KLayout.
    layer_index, text_magnification: int
        KLayout layer number and text size.
    dose: float
        Text layer dose.
    text_identifier, text_string: string
        Text cell name and text to write.

    Returns
    -------
    text_cell: object
        Text cell.

    See Also
    --------
    makes_bars_cell

    Notes
    -----
    None

    Example
    -------
    None

    """
    text_layer = layout.layer(layer_index, dose)
    text_cell = layout.create_cell(f'{text_identifier}')
    generator = db.TextGenerator.default_generator()
    region = generator.text(
        text_string,
        layout.dbu,
        text_magnification)
    text_cell.shapes(text_layer).insert(region)
    return text_cell


def makes_gratings_cell(layout : object,
                        grating_bar : object,
                        period : float,
                        grating_length : float,
                        grating_identifier : str) -> object:
    """
    Create a grating cell with the given bar cell, period, and grating length.

    Parameters
    ----------
    layout, grating_bar: object
        Database and cell objects from KLayout.
    period, grating_length: float
        Period or the grating and grating length in database units.
    grating_identifier: string
        Grating cell name.
    
    Returns
    -------
    grating_cell: object
        Grating cell.
    
    See Also
    --------
    makes_bars_cell

    Notes
    -----
    This makes a 1D grating with a period in X.

    Example
    -------
    None

    """
    grating_cell = layout.create_cell(f'{grating_identifier}')
    x_vector = db.DVector(period, 0)
    num_x = math.floor(grating_length / period)
    y_vector = db.DVector()
    num_y = 1
    grating_cell.insert(
        db.DCellInstArray(
            grating_bar.cell_index(),
            db.DTrans(),
            x_vector,
            y_vector,
            num_x,
            num_y))
    return grating_cell


def makes_nanohole_cell(layout : object,
                        hole_cell : object,
                        period_x : float,
                        period_y : float,
                        grating_length : float,
                        grating_height : float,
                        grating_identifier : str) -> object:
    """
    Create a nanohole cell with the given hole cell, period, and grating length.

    Parameters
    ----------
    layout, grating_bar: object
        Database and cell objects from KLayout.
    period_x, period_y, grating_length, grating_height: float
        Period of the grating and grating length in database units.
    grating_identifier: string
        Grating cell name.
    
    Returns
    -------
    grating_cell: object
        Grating cell.
    
    See Also
    --------
    makes_hole_cell

    Notes
    -----
    This makes a 2D grating with a period in X and Y.

    Example
    -------
    None

    """
    grating_cell = layout.create_cell(f'{grating_identifier}')
    x_vector = db.DVector(period_x, 0)
    num_x = math.floor(grating_length / period_x)
    y_vector = db.DVector(0, period_y)
    num_y = math.floor(grating_height / period_y)
    grating_cell.insert(
        db.DCellInstArray(
            hole_cell.cell_index(),
            db.DTrans(),
            x_vector,
            y_vector,
            num_x,
            num_y))
    return grating_cell


def chirped_coordinates(periods : list,
                        heights : list,
                        writefield_width : float) -> list:
    """
    Parameters
    ----------
    periods, heights: list
        Grating periods at the extreme left, central, and extreme right. Grating
        heights at the same values.

    Returns
    -------
    x_coords, y_coords: list
        List of x and y coordinates for the chirped bars.

    See Also
    --------
    None

    Notes
    -----
    None

    Example
    -------
    None

    """
    number_periods = math.floor(writefield_width / max(periods))
    x_central = []
    for period in periods:
        temporary = np.arange(
            - number_periods / 2 * period,
            (number_periods / 2 * period) + period,
            period)
        x_central.append(list(temporary))
    x_cen = list(
        np.asarray(x_central).reshape(len(periods), -2).transpose())
    x_coords = []
    y_coords = []
    for x_c in x_cen:
        x_c = list(x_c)
        temporary_x = []
        temporary_y = []
        for x, p, h in zip(x_c, periods, heights):
            temporary_x.append(x - (p / 2))
            temporary_y.append(h)
        for x, p, h in zip(reversed(x_c), reversed(periods), reversed(heights)):
            temporary_x.append(x - (p / 2) + (p * (1 - ff)))
            temporary_y.append(h)
        x_coords.append(temporary_x)
        y_coords.append(temporary_y)
    return x_coords, y_coords


def make_chirpedgrating(layout : object,
                        layer : object,
                        periods : list,
                        heights : list,
                        writefield_width : float,
                        grating_identifier : str) -> object:
    """
    Parameters
    ----------
    layout, layer: object
        Database and layer objected from KLayout.
    periods, heights: list
        Grating periods and heights for grating extremes.
    writefield_width: float
        Writefield width in database units.
    grating_identifier: str
        Grating cell name

    Returns
    -------
    grating_cell: object
        Grating cell.

    See Also
    --------
    chirped_coordinates

    Notes
    -----
    None

    Example
    -------
    None

    """
    grating_cell = layout.create_cell(f'{grating_identifier}')
    x_coordinates, y_coordinates = chirped_coordinates(
        periods=periods,
        heights=heights,
        writefield_width=writefield_width)
    for x, y in zip(x_coordinates, y_coordinates):
        points = []
        for point in zip(x, y):
            points.append(db.DPoint(*point))
        grating_cell.shapes(layer).insert(db.DPolygon(points))
    return grating_cell

## Database Design

* We know the periods, fill factors, etc for this chip from the previous design, and I will keep things constant again for now.
* The first thing we can do is set up the chip outline, this layer will not have anything written on it, but it will allow us to see where on the chip our patterns will appear.

### Chip Outline

* Create the zero dose chip outline layer.

In [71]:
chip_cell = layout.create_cell('Chip_Outline')
layer_index = 0
layer = layout.layer(layer_index, 0)

* Now we draw the objects based on the above chip outline dimensions.

In [72]:
chip_bottom = db.DBox(
    db.DPoint(0, 0),
    (db.DPoint(0, 0) + db.DVector(chip_width, - 100)))
chip_left = db.DBox(
    db.DPoint(0, 0),
    (db.DPoint(0, 0) + db.DVector(-100, chip_height)))
chip_right = db.DBox(
    db.DPoint(chip_width, 0),
    (db.DPoint(chip_width, 0) + db.DVector(100, chip_height)))
chip_top = db.DBox(
    db.DPoint(0, chip_height),
    (db.DPoint(0, chip_height) + db.DVector(chip_width, 100)))

* Finally, add the chip outline boxes to the chip_cell.

In [73]:
chip_cell.shapes(layer).insert(chip_bottom)
chip_cell.shapes(layer).insert(chip_left)
chip_cell.shapes(layer).insert(chip_right)
chip_cell.shapes(layer).insert(chip_top)

box (0,15000000;15000000,15100000)

* In the future, this can easily be a function, because the chip outline will always be the same.

### Gratings Dictionary

* Add the grating dictionaries at the top of the database.

In [74]:
gratings_dictionary = {}

### Small Period Gratings

* Let's start with the small period gratings.
* We are going to use a bunch of different gratings with different periods and different dose factor scalars.
* So we start by setting up those ranges.

In [75]:
dose_factors = np.arange(1, 1.51, 0.5)
periods = range(300, 381, 20)
fill_factors = 0.7
grating_spacing = 1000
text_magnification = 100
text_spacing = 75

* Now we set up the dose cells.
* The dose cells will contain rows of gratings.
* Remember that we need to scale the dose factor by 1000.
* Note that when are replicating the same shape, then we use a different instance of DCellInstArray.
* When we build the dose row, we instead use the DTrans(DVector) method to space them out.
* This mildly increases the file size, but should not be close to that of the previous iteration.

In [76]:
layer_index = 1
text_layer = layout.layer(len(dose_factors) + 1, 2000)
flat_cell1 = layout.create_cell('Low_P_Gratings')
for i, dose in enumerate(dose_factors):
    layer = layout.layer(layer_index + i, dose * 1000)
    dose_cell = layout.create_cell(f'LowPt_df{dose}')
    dose_text_cell = layout.create_cell(f'LowP_df{dose}')
    for j, period in enumerate(periods):
        grating_period = period / 1000  # scale to database units
        grating_identifier = f'{sample_identifier}.lowp.{i}.{j}'
        gratings_dictionary.update(
            {grating_identifier: f'df{dose}_p{period}_ff{fill_factors}'})
        bar_cell = makes_bars_cell(
            layout=layout,
            layer=layer,
            period=grating_period,
            fill_factor=fill_factors,
            bar_height=writefield_height,
            bar_identifier=f'LowPBar_df{dose}_p{period}_ff{fill_factors}')
        grating_cell = makes_gratings_cell(
            layout=layout,
            grating_bar=bar_cell,
            period=grating_period,
            grating_length=writefield_width,
            grating_identifier=(
                f'LowP_df{dose}_p{period}_ff{fill_factors}'))
        text_cell = generates_texts(
            layout=layout,
            layer_index=len(dose_factors) + 1,
            dose=2000,
            text_identifier=f'LowPText_df{dose}_p{period}_ff{fill_factors}',
            text_string=grating_identifier,
            text_magnification=text_magnification)
        dose_cell.insert(
            db.DCellInstArray(
                grating_cell.cell_index(),
                db.DTrans(
                    db.DVector(
                        j * (writefield_width + grating_spacing),
                        0))))
        dose_text_cell.insert(
            db.DCellInstArray(
                text_cell.cell_index(),
                db.DTrans(
                    db.DVector(
                        j * (writefield_width + grating_spacing),
                        writefield_height + text_spacing))))
    flat_cell1.insert(
        db.DCellInstArray(
            dose_cell.cell_index(),
            db.DTrans(
                db.DVector(
                    0,
                    i * (writefield_height + grating_spacing)))))
    flat_cell1.insert(
        db.DCellInstArray(
            dose_text_cell.cell_index(),
            db.DTrans(
                db.DVector(
                    0,
                    i * (writefield_height + grating_spacing)))))

### Medium Period Gratings

* Let's start with the medium period gratings.
* We are going to use a bunch of different gratings with different periods and different dose factor scalars.
* So we start by setting up those ranges.

In [77]:
dose_factors = np.arange(1, 1.51, 0.5)
periods = range(400, 481, 20)
fill_factors = 0.7
grating_spacing = 1000
text_magnification = 100
text_spacing = 75
layer_index = 1
text_layer = layout.layer(len(dose_factors) + 1, 2000)
flat_cell2 = layout.create_cell('Med_P_Gratings')
for i, dose in enumerate(dose_factors):
    layer = layout.layer(layer_index + i, dose * 1000)
    dose_cell = layout.create_cell(f'MedPt_df{dose}')
    dose_text_cell = layout.create_cell(f'MedP_df{dose}')
    for j, period in enumerate(periods):
        grating_period = period / 1000  # scale to database units
        grating_identifier = f'{sample_identifier}.medp.{i}.{j}'
        gratings_dictionary.update(
            {grating_identifier: f'df{dose}_p{period}_ff{fill_factors}'})
        bar_cell = makes_bars_cell(
            layout=layout,
            layer=layer,
            period=grating_period,
            fill_factor=fill_factors,
            bar_height=writefield_height,
            bar_identifier=f'MedPBar_df{dose}_p{period}_ff{fill_factors}')
        grating_cell = makes_gratings_cell(
            layout=layout,
            grating_bar=bar_cell,
            period=grating_period,
            grating_length=writefield_width,
            grating_identifier=(
                f'MedP_df{dose}_p{period}_ff{fill_factors}'))
        text_cell = generates_texts(
            layout=layout,
            layer_index=len(dose_factors) + 1,
            dose=2000,
            text_identifier=f'MedPText_df{dose}_p{period}_ff{fill_factors}',
            text_string=grating_identifier,
            text_magnification=text_magnification)
        dose_cell.insert(
            db.DCellInstArray(
                grating_cell.cell_index(),
                db.DTrans(
                    db.DVector(
                        j * (writefield_width + grating_spacing),
                        0))))
        dose_text_cell.insert(
            db.DCellInstArray(
                text_cell.cell_index(),
                db.DTrans(
                    db.DVector(
                        j * (writefield_width + grating_spacing),
                        writefield_height + text_spacing))))
    flat_cell2.insert(
        db.DCellInstArray(
            dose_cell.cell_index(),
            db.DTrans(
                db.DVector(
                    0,
                    i * (writefield_height + grating_spacing)))))
    flat_cell2.insert(
        db.DCellInstArray(
            dose_text_cell.cell_index(),
            db.DTrans(
                db.DVector(
                    0,
                    i * (writefield_height + grating_spacing)))))

### Large Period Gratings

* Let's start with the large period gratings.
* We are going to use a bunch of different gratings with different periods and different dose factor scalars.
* So we start by setting up those ranges.

In [78]:
dose_factors = np.arange(1, 1.51, 0.5)
periods = range(500, 581, 20)
fill_factors = 0.7
grating_spacing = 1000
text_magnification = 100
text_spacing = 75
layer_index = 1
text_layer = layout.layer(len(dose_factors) + 1, 2000)
flat_cell3 = layout.create_cell('High_P_Gratings')
for i, dose in enumerate(dose_factors):
    layer = layout.layer(layer_index + i, dose * 1000)
    dose_cell = layout.create_cell(f'HighPt_df{dose}')
    dose_text_cell = layout.create_cell(f'HighP_df{dose}')
    for j, period in enumerate(periods):
        grating_period = period / 1000  # scale to database units
        grating_identifier = f'{sample_identifier}.highp.{i}.{j}'
        gratings_dictionary.update(
            {grating_identifier: f'df{dose}_p{period}_ff{fill_factors}'})
        bar_cell = makes_bars_cell(
            layout=layout,
            layer=layer,
            period=grating_period,
            fill_factor=fill_factors,
            bar_height=writefield_height,
            bar_identifier=f'HighPBar_df{dose}_p{period}_ff{fill_factors}')
        grating_cell = makes_gratings_cell(
            layout=layout,
            grating_bar=bar_cell,
            period=grating_period,
            grating_length=writefield_width,
            grating_identifier=(
                f'HighP_df{dose}_p{period}_ff{fill_factors}'))
        text_cell = generates_texts(
            layout=layout,
            layer_index=len(dose_factors) + 1,
            dose=2000,
            text_identifier=f'HighPText_df{dose}_p{period}_ff{fill_factors}',
            text_string=grating_identifier,
            text_magnification=text_magnification)
        dose_cell.insert(
            db.DCellInstArray(
                grating_cell.cell_index(),
                db.DTrans(
                    db.DVector(
                        j * (writefield_width + grating_spacing),
                        0))))
        dose_text_cell.insert(
            db.DCellInstArray(
                text_cell.cell_index(),
                db.DTrans(
                    db.DVector(
                        j * (writefield_width + grating_spacing),
                        writefield_height + text_spacing))))
    flat_cell3.insert(
        db.DCellInstArray(
            dose_cell.cell_index(),
            db.DTrans(
                db.DVector(
                    0,
                    i * (writefield_height + grating_spacing)))))
    flat_cell3.insert(
        db.DCellInstArray(
            dose_text_cell.cell_index(),
            db.DTrans(
                db.DVector(
                    0,
                    i * (writefield_height + grating_spacing)))))

## Patterning the Chip

* This bit can be done either within KLayout or on the chip.
* Given that the overall file size before this part is just under 2500 KB, I am going to pattern the chip and load one singular file into the voyager system.
* The system can handle the individual database elements too, so having a "position list" is not a bad thing either.
* We start by adding the cells to the top cell.
* The cell names are:
  * flat_cell1
  * flat_cell2
  * flat_cell3
  * chip_cell

In [79]:
x_shift = 3000
y_shift = 2000

transform_0 = db.ICplxTrans(1, 0, False, 0, 0)
transform_1 = db.ICplxTrans(1, 0, False, x_shift, y_shift)

transform_2 = db.ICplxTrans(1, 0, False, x_shift, y_shift + 4 * grating_spacing)

transform_3 = db.ICplxTrans(1, 0, False, x_shift, y_shift + 8 * grating_spacing)
transform_4 = db.ICplxTrans(1, 0, False, x_shift, y_shift + 16 * grating_spacing)

top_cell.insert(
    db.DCellInstArray(
        chip_cell.cell_index(),
        transform_0))
top_cell.insert(
    db.DCellInstArray(
        flat_cell1.cell_index(),
        transform_1))
top_cell.insert(
    db.DCellInstArray(
        flat_cell2.cell_index(),
        transform_2))
top_cell.insert(
    db.DCellInstArray(
        flat_cell3.cell_index(),
        transform_3))

cell_index=72 r0 3000000,10000000

## Grating Keys

* Just the grating keys to output, so that each of the grating numbers on the chips can be easily identified.
* To reduce the writing time on the chip, this can simply be printed here.

In [80]:
display(
    Markdown(
        '\n'.join(
            [
                f'{key}: {value}\n'
                for key, value
                in gratings_dictionary.items()])))

AR4.lowp.0.0: df1.0_p300_ff0.7

AR4.lowp.0.1: df1.0_p320_ff0.7

AR4.lowp.0.2: df1.0_p340_ff0.7

AR4.lowp.0.3: df1.0_p360_ff0.7

AR4.lowp.0.4: df1.0_p380_ff0.7

AR4.lowp.1.0: df1.5_p300_ff0.7

AR4.lowp.1.1: df1.5_p320_ff0.7

AR4.lowp.1.2: df1.5_p340_ff0.7

AR4.lowp.1.3: df1.5_p360_ff0.7

AR4.lowp.1.4: df1.5_p380_ff0.7

AR4.medp.0.0: df1.0_p400_ff0.7

AR4.medp.0.1: df1.0_p420_ff0.7

AR4.medp.0.2: df1.0_p440_ff0.7

AR4.medp.0.3: df1.0_p460_ff0.7

AR4.medp.0.4: df1.0_p480_ff0.7

AR4.medp.1.0: df1.5_p400_ff0.7

AR4.medp.1.1: df1.5_p420_ff0.7

AR4.medp.1.2: df1.5_p440_ff0.7

AR4.medp.1.3: df1.5_p460_ff0.7

AR4.medp.1.4: df1.5_p480_ff0.7

AR4.highp.0.0: df1.0_p500_ff0.7

AR4.highp.0.1: df1.0_p520_ff0.7

AR4.highp.0.2: df1.0_p540_ff0.7

AR4.highp.0.3: df1.0_p560_ff0.7

AR4.highp.0.4: df1.0_p580_ff0.7

AR4.highp.1.0: df1.5_p500_ff0.7

AR4.highp.1.1: df1.5_p520_ff0.7

AR4.highp.1.2: df1.5_p540_ff0.7

AR4.highp.1.3: df1.5_p560_ff0.7

AR4.highp.1.4: df1.5_p580_ff0.7


## Database Write Out

* Finish by writing out the database file.

In [81]:
layout.write(f'{out_path}/{sample_identifier}_231109.gds')